#### This notebook is for IBM certificate capstone project on Coursera.

In [10]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup as bs

#### Here I am using Requests library to retrieve webpage from wikipedia

In [26]:
resp = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
pageText = resp.text
soup = bs(pageText, 'html.parser')

In [29]:
table = soup.find_all("table")

In [49]:
tablist = []
for mytable in table:
    table_body = mytable.find('tbody')
    try:
        rows = table_body.find_all('tr')
        for tr in rows:
            cols = tr.find_all('td')
            temp = ""
            for td in cols:
                temp += td.text+","
            tablist.append(temp)
    except:
        print("no tbody")

no tbody


There is some cluttering in start and after of the table while scraping so removing them by using indexing...

In [50]:
tablist = tablist[1:-7]

In [59]:
## Here each row is being put into separate list. This will make is easy to import to dataframe
## in separate columns.

new_tablist = []
for each in tablist:
    each = each.replace('\n','')
    new_tablist.append(each.split(','))


In [81]:
db = pd.DataFrame(new_tablist)

In [82]:
## Giving columns names, the last one is extraneous columns, so will drop it
db.columns = ['PostalCode', 'Borough', 'Neighbourhood','todrop']

In [83]:
db.drop('todrop',axis=1,inplace=True)

In [84]:
## Ignoring those postal areas with Borough = Not assigned
db = db[db['Borough']!='Not assigned']

In [85]:
db.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [86]:
## see the index is disturbed after, so resetting it and dropping the previous index columns
db.reset_index(inplace=True)
db.drop('index',axis=1,inplace=True)

In [140]:
## we need onyl unique PostalCode and want to merge the duplicate Neighbourhood.
db2 = db.groupby(['PostalCode']).agg({'Borough':'first', 'Neighbourhood': ', '.join}).reset_index()

In [141]:
db2.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [142]:
## Checking if our logic was right and the answer shows that two neighbour are merged correctly
db2[db2['PostalCode']=='M5A']

,PostalCode,Borough,Neighbourhood
53,M5A,Downtown Toronto,"Harbourfront, Regent Park"


In [143]:
## Checking if there are such PostalCodes with Neighbourhood set to Not assigned
db2[db2['Neighbourhood']=='Not assigned']

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


In [144]:
## np.where is a good option to replace, see the documentation
db2['Neighbourhood'] = np.where(db2['Neighbourhood']!='Not assigned', db2['Neighbourhood'], db2['Borough'])

In [145]:
## checking if our logic was right, turns out that Neighbourhood is correctly replaced by Borough value
db2[db2['PostalCode']=='M7A']

,PostalCode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


In [146]:
db2.shape

(103, 3)